In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
top_100_url = "https://tophotels.ru/club/ratings/top100"
r = requests.get(top_100_url)
soup = BeautifulSoup(r.text, "html.parser")

In [3]:
top_10_list = soup.find_all("table")[0].find_all('tr')[1:3]

In [4]:
hotels = dict()

for item in top_10_list:
    hotels[item.a.get_text().strip()] = item.a.attrs['href']

hotels

{'Maxx Royal Belek Golf Resort 5* \nСПА-отель': '/hotel/al61593',
 'Lujo Bodrum 5* \nОтель': '/hotel/al313904'}

###Reviews

In [5]:
import requests
from bs4 import BeautifulSoup
import time

hotels_reviews = {}
num = 0
for hotel_name, href in hotels.items():
    page_number = 1
    hotel_reviews = []
    while page_number!=2 :
        url = f'https://tophotels.ru{href}/reviews/list?page={page_number}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        current_page_reviews = soup.find_all("div", class_="review__txt-wrap")
        if not current_page_reviews:
            break

        for review in current_page_reviews:
            hotel_reviews.append(review.get_text().strip())

        page_number += 1
        time.sleep(1)
    num += 1
    print(num)
    hotels_reviews[hotel_name] = hotel_reviews


for hotel_name, reviews in hotels_reviews.items():
    print(f"Total reviews collected for {hotel_name}: {len(reviews)}")


1
2
Total reviews collected for Maxx Royal Belek Golf Resort 5* 
СПА-отель: 21
Total reviews collected for Lujo Bodrum 5* 
Отель: 22


In [6]:
first_hotel_name, first_hotel_reviews = next(iter(hotels_reviews.items()))
print(first_hotel_name, first_hotel_reviews)

Maxx Royal Belek Golf Resort 5* 
СПА-отель ['Прекрасный Макс Рояль!\nДобрый вечер, была в Максе (Белек)во второй раз, поэтому буду сравнивать с отдыхом в 2021 г.\r\nПерелет туркишами прошел отлично, трансфер был индивидуальный поэтому до отеля доехала очень быстро. Встреча на ресепшене асисстентами, красная икра :), все как всегда на высоте.Так как приехала рано утром, номер был не готов, отправили на завтрак,потом погуляла по территори и в часов 11 заселили....', 'Подарить себе праздник\nНе всегда  удается оперативно писать отзывы о посещаемых отелях, но с другой стороны было бы неправильно вообще не упоминать об этом туре, тем более мой рассказ о поездке в несезон, а он как раз и наступает вместе с  чередой  всевозможных праздничных дней и наверняка  мои впечатления и фото помогут туристам  сделать свой выбор.\n\r\nИтак, условия по которым я выбирала  отель для холодного...', 'Еще один любимый отель.\nОтдыхали в октябре семьей. Ранее отдыхали в Белеке, но в Риксосе. Решили сменить об

In [7]:
import requests
from bs4 import BeautifulSoup
import time

hotels_info = {}
miss_desc = []
for hotel_name, href in hotels.items():
    try:
        hotels_info[hotel_name] = {"description": None, "reviews": []}


        desc_url = f'https://tophotels.ru{href}'
        r = requests.get(desc_url)
        print(f"Fetching description for {hotel_name}: {r}")
        soup = BeautifulSoup(r.text, "html.parser")
        divs = soup.find_all('div', {"class": "grey-item__white"})

        if divs:
            hotel_desc = divs[0].get_text().strip()
            hotels_info[hotel_name]["description"] = hotel_desc
        else:
            miss_desc.append(hotel_name)

        page_number = 1
        while page_number != 2:
            review_url = f'https://tophotels.ru{href}/reviews/list?page={page_number}'
            response = requests.get(review_url)
            soup = BeautifulSoup(response.text, "html.parser")

            current_page_reviews = soup.find_all("div", class_="review__txt-wrap")
            if not current_page_reviews:
                break

            for review in current_page_reviews:
                hotels_info[hotel_name]["reviews"].append(review.get_text().strip())

            page_number += 1
            time.sleep(1)

    except Exception as e:
        print(f"Error processing {hotel_name}: {e}")



Fetching description for Maxx Royal Belek Golf Resort 5* 
СПА-отель: <Response [200]>
Fetching description for Lujo Bodrum 5* 
Отель: <Response [200]>


In [8]:
for hotel_name, info in hotels_info.items():
    combined_text = info['description'] + " " + " ".join(info['reviews'])
    hotels_info[hotel_name]['combined_text'] = combined_text

In [9]:
first_hotel_name, first_hotel_reviews = next(iter(hotels_info.items()))

print(first_hotel_name, first_hotel_reviews)

Maxx Royal Belek Golf Resort 5* 
СПА-отель {'description': 'Месторасположение: \r\nОткрытый в 2011 году Maxx Royal Belek Golf Resort расположен в одном из наиболее изысканных уголков Белека, в непосредственной близости к центру города и транспортным путям, в 35 км от аэропорта, в 45 км от центра города Анталья и всего в 1 км от города Белек.\r\n\xa0\r\nОб отеле:\r\nОтель Maxx Royal Belek Golf Resortпредлагает Вам окунуться в атмосферу роскоши и комфорта. Наш квалифицированный персонал поможет сделать Ваш отдых незабываемым, и всегда готовы прийти на помощь. Также Вы оцените роскошное размещение , от сьютов до вилл Монтгомери, превосходящеговсе Ваши ожидания.', 'reviews': ['Прекрасный Макс Рояль!\nДобрый вечер, была в Максе (Белек)во второй раз, поэтому буду сравнивать с отдыхом в 2021 г.\r\nПерелет туркишами прошел отлично, трансфер был индивидуальный поэтому до отеля доехала очень быстро. Встреча на ресепшене асисстентами, красная икра :), все как всегда на высоте.Так как приехала ран

###  


In [10]:
import os
!pip install openai
!pip install langchain
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = "sk-wIcqz35tV9EHW6aR9lMHQVYmMOq30iAI"
os.environ['OPENAI_API_BASE'] = "https://api.proxyapi.ru/openai/v1"
!pip install langchain_openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.5/111.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging

In [11]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00


In [12]:
embeddings = OpenAIEmbeddings()
docs = [
    Document(page_content=info['combined_text'], metadata={"name": hotel_name})
    for hotel_name, info in hotels_info.items()
    if 'combined_text' in info
]
vector = FAISS.from_documents(docs, embeddings)

In [13]:
vector

In [14]:
vector.save_local("faiss_index")

#new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [15]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

### saving db

In [16]:
import json

filename = 'hotels_info.json'

with open(filename, 'w', encoding='utf-8') as f:
    json.dump(hotels_info, f, ensure_ascii=False, indent=4)

In [18]:
from IPython.display import FileLink

display(FileLink(filename))

/content/hotels_info.json

In [22]:
vector.save_local("faiss_index")

TypeError: 'IndexFlatL2' object is not callable

In [ ]:
import pickle

# Save the document vectors to a file
with open("document_vectors.pkl", "wb") as f:
    pickle.dump(docs, f)

In [ ]:
np.save('embeddings.npy', embeddings)

In [ ]:
display(FileLink('hotels_info.json'))
display(FileLink('hotels_index.faiss'))
display(FileLink('embeddings.npy'))

### Start

In [54]:
query = "Мне нужен отель со спортивными активностями и вкусной едой, при этом мне безразлично наличие и близость к морю" # -> vector
docs_and_scores =  vector.similarity_search_with_score(query) # ->

In [55]:
for n, (doc, score) in enumerate(docs_and_scores):
    print(f"{n+1}) Hotel {doc.metadata['name'].split()[0]} with score {score:.2f}")
    #print(doc.page_content)

1) Hotel Lujo with score 0.33
2) Hotel Maxx with score 0.37


In [39]:
llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template("""Помоги пользователю подобрать отель под его запрос:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [40]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [56]:
from langchain.schema import StrOutputParser
template = """Помоги пользователю подобрать отель под его запрос:

{context}

Question: {input}
"""
human_template = "{text}"

# Create the template from the messages
prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

# Create a chain of operations to process the input and generate reviews
summary_chain = prompt | llm | StrOutputParser()

# Invoke the summary chain to generate reviews
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "Мне нужен отель на берегу моря с хорошей кухней"})
print(response["answer"])
#res = summary_chain.invoke({'text': 'I want perfect beach but food does not matter'})
#print(res)

Отель Maxx Royal Belek Golf Resort может быть отличным выбором для вас. Здесь вы найдете превосходное обслуживание, разнообразную и вкусную еду во всех ресторанах, а также уютные номера, качественный сервис и красивые виды на море. Кроме того, отель расположен на берегу моря, что добавит удовольствия к вашему отдыху.


In [42]:
response = retrieval_chain.invoke({"input": "Мне нужен отель на берегу моря с хорошей кухней"})
print(response["answer"])

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 36951 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
response

{'input': 'Мне нужен отель на берегу моря с хорошей кухней',
 'context': [Document(page_content='Больше простора, больше уюта, больше свободы, больше креатива... Необыкновенная гармония бескрайней морской бирюзы, ласкового солнца и удивительной природы... Умные и роскошные номера Lara Barut Collection предлагают провести умиротворенный, веселый и динамичный отдых!', metadata={'name': 'Lara Barut Collection 5* \nОтель'}),
  Document(page_content='Отель - курорт"SAVOY"- идеальное место для отдыха. Флора и фауна, водопады и живописные пейзажи не оставят Вас равнодушными. Неподалёку расположен заповедник мирового значения Рас Мухаммед. Тропический ландшафт, и вечнозелёная растительность - настоящий оазис, расположившийся между горными массивами и девственно чистым берегом Красного моря.\n\nМесторасположение:\nОтель расположен на берегу моря, в бухте "White Knight Beach" и шаговой доступности от развлекательного комплекса Soho.В8 км от аэропортаи 15 км от Naama Bay.', metadata={'name': 'Sav

### new combined


In [24]:
from langchain.schema import StrOutputParser
llm = ChatOpenAI()
template = """Help the user find a hotel based on their request:
Question: {text}
"""
human_template = "{text}"

# Assuming ChatPromptTemplate is something you can define or adjust
prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
summary_chain = prompt | llm | StrOutputParser()
input_request = "I need a hotel on the beach with good food"

In [27]:
llm_response = summary_chain.invoke({'text': input_request})
print("LLM Response:", llm_response)
refined_query = llm_response
try:
    docs_and_scores = vector.similarity_search_with_score(refined_query)
    for n, (doc, score) in enumerate(docs_and_scores):
        print(f"{n+1}) Hotel {doc.metadata['name'].split()[0]} with score {score:.2f}")
except Exception as e:
    print(f"Error during vector search: {e}")

LLM Response: I recommend looking into beachfront hotels that offer dining options with positive reviews. Here are a few suggestions:

1. The Ritz-Carlton, Cancun - This luxurious hotel is located on a beautiful beach in Cancun, Mexico and offers several dining options with gourmet cuisine.

2. Four Seasons Resort Maui at Wailea - Situated on a stunning beach in Maui, Hawaii, this resort features top-notch restaurants serving delicious meals with ocean views.

3. Grand Velas Riviera Maya - Located on a pristine beach in the Riviera Maya, Mexico, this all-inclusive resort offers a variety of dining options, including gourmet restaurants.

4. The Breakers Palm Beach - This historic hotel in Palm Beach, Florida has a prime beachfront location and multiple dining venues known for their excellent cuisine.

Remember to check the hotel's website or online reviews to ensure they meet your expectations for both beach access and food quality. Enjoy your stay!
1) Hotel Maxx with score 0.48
2) Hot

In [40]:
llm = ChatOpenAI()
template = "Help the user find a hotel based on their request:\nQuestion: {text}"
human_template = "{text}"

# Assuming ChatPromptTemplate can be set like this
prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

# Chain setup
summary_chain = prompt | llm | StrOutputParser()

# Input request
input_request = "I need a hotel on the beach with good food"

# Invoke chain to get refined query
llm_response = summary_chain.invoke({'text': input_request})
print("LLM Response:", llm_response)


LLM Response: I recommend looking into beachfront resorts or hotels in popular beach destinations. Some options to consider are:

1. **Four Seasons Resort Maui at Wailea** (Hawaii) - This luxurious resort is located on a beautiful beach in Maui and offers excellent dining options.
   
2. **The Ritz-Carlton, Cancun** (Mexico) - Situated on a stunning beach in Cancun, this hotel is known for its upscale accommodations and delicious dining choices.
   
3. **JW Marriott Guanacaste Resort & Spa** (Costa Rica) - This beachfront resort in Costa Rica offers a relaxing atmosphere and a variety of dining options with a focus on fresh, local ingredients.
   
4. **Soneva Kiri** (Thailand) - Located on a private beach in Koh Kood, this luxury resort offers a unique dining experience with sustainable and organic cuisine.
   
5. **Banyan Tree Seychelles** (Seychelles) - Nestled on the picturesque beach of Anse Intendance, this resort provides guests with a culinary journey through its various dining 

In [42]:
retriever = vector.as_retriever()
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
input_request = "your query here"  # Define your input query
response = retrieval_chain.invoke({"input": input_request})
# Get the initial set of documents based on the input query
# Using the retrieved documents, refine the query or process further
refined_query = " ".join(doc.text for doc in initial_docs)  # Example of refining the query

try:
    # Further retrieval based on the refined query
    docs_and_scores = faiss_retriever.retrieve_with_score(refined_query, top_k=5)  # Retrieve top 5 documents with scores
    for n, (doc, score) in enumerate(docs_and_scores):
        print(f"{n+1}) Hotel {doc.metadata['name'].split()[0]} with score {score:.2f}")
except Exception as e:
    print(f"Error during FAISS retrieval: {e}")


ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['text']